In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from langchain import hub

prompt_agent = hub.pull('hwchase17/openai-tools-agent')

prompt_agent

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('/Users/jasper/Downloads/My_first_agents-main/notebooks/.env')

# Verify that the API key is loaded
api_key = os.getenv('OPENAI_API_KEY')

In [6]:
#import os
#from dotenv import load_dotenv

# Load environment variables from .env file
#load_dotenv('/Users/jasper/Downloads/My_first_agents-main/notebooks/.env')

# Verify that the API key is loaded
tavily_api_key = os.getenv('TAVILY_API_KEY')

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0, openai_api_key=api_key)

llm.invoke("HI")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-07e7eb52-e53d-445a-93f7-6da6798149c6-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Agent = LLM + Tools

In [8]:
import subprocess
from langchain_core.tools import tool


@tool
def list_files():
    """List files in current directory"""
    return subprocess.run(["ls"], stdout=subprocess.PIPE).stdout.decode("utf-8")


list_files.invoke({"input": ""})

'1.0-langchain-basics.ipynb\n1.1-langchain-simple-chains.ipynb\n1.2-langchain-runnables-and-lcel.ipynb\n1.3-langchain-simple-agent.ipynb\n2.0-autogen-building-agents.ipynb\n2.1-autogen-building-research-assistant.ipynb\n3.0-langgraph-building-stateful-applications.ipynb\n3.1-langgraph-buid-research-assistant.ipynb\n4.0-rag-agent-langgraph.ipynb\n4.1-autogen-multi-agent-workflows.ipynb\nAutogen Demo Notebook.ipynb\nTool Use.ipynb\nagentic-rag-langgraph.ipynb\n\x1b\x1bassets-resources\x1b[m\x1b[m\n'

In [9]:
llm_with_tool = llm.bind_tools([list_files])

llm_with_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1088f4190>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10893a2d0>, root_client=<openai.OpenAI object at 0x1088cfc10>, root_async_client=<openai.AsyncOpenAI object at 0x10891c890>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'list_files', 'description': 'List files in current directory', 'parameters': {'properties': {}, 'type': 'object'}}}]}, config={}, config_factories=[])

In [10]:
llm_with_tool.invoke("List files in current directory")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rvyEIE5amrHVqjTkWnEsxad0', 'function': {'arguments': '{}', 'name': 'list_files'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 42, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4ab78625-a55a-4539-a331-b1875e3b1dfd-0', tool_calls=[{'name': 'list_files', 'args': {}, 'id': 'call_rvyEIE5amrHVqjTkWnEsxad0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 10, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
dictionary_with_tool_names = {
    "list_files": list_files,
}

output = llm_with_tool.invoke("List files in current directory")

if output.tool_calls:
    for tool_call in output.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call['args']
        tool_func = dictionary_with_tool_names[tool_name]
        print(tool_func.invoke(tool_args))

1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
Autogen Demo Notebook.ipynb
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources



Agent = LLM + tools

In [12]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

/opt/homebrew/anaconda3/envs/vidhya-agents/lib/python3.11/site-packages/langchain/chains/api/base.py:56: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.utilities.requests import TextRequestsWrapper


In [13]:
tools = [list_files]

agent = create_tool_calling_agent(llm, tools, prompt_agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.invoke({'input': "List files in current directory"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
Autogen Demo Notebook.ipynb
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources
Here are the files in the current directory:

1. 1.0-langchain-basics.ipynb
2. 1.1-langchain-simple-chains.ipynb
3. 1.2-langchain-runnables-and-lcel.ipynb
4. 1.3-langchain-simple-agent.ipynb
5. 2.0-autogen-building-agents.ipynb
6. 2.1-autogen-building-research-assistant.ipynb
7. 3.0-langgraph-building-stateful-applications.ipynb
8. 3.1-langgraph-buid-research-assistant.ipynb
9. 4.0-rag-agent-langgraph.ipynb
10. 4.1-autogen-multi-agent-workflows.ipynb
11.

{'input': 'List files in current directory',
 'output': 'Here are the files in the current directory:\n\n1. 1.0-langchain-basics.ipynb\n2. 1.1-langchain-simple-chains.ipynb\n3. 1.2-langchain-runnables-and-lcel.ipynb\n4. 1.3-langchain-simple-agent.ipynb\n5. 2.0-autogen-building-agents.ipynb\n6. 2.1-autogen-building-research-assistant.ipynb\n7. 3.0-langgraph-building-stateful-applications.ipynb\n8. 3.1-langgraph-buid-research-assistant.ipynb\n9. 4.0-rag-agent-langgraph.ipynb\n10. 4.1-autogen-multi-agent-workflows.ipynb\n11. Autogen Demo Notebook.ipynb\n12. Tool Use.ipynb\n13. agentic-rag-langgraph.ipynb\n14. assets-resources (directory)'}

In [15]:
agent_executor.invoke({'input': "What kind of files are in the current directory?"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
Autogen Demo Notebook.ipynb
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources
The current directory contains the following files:

1. `1.0-langchain-basics.ipynb`
2. `1.1-langchain-simple-chains.ipynb`
3. `1.2-langchain-runnables-and-lcel.ipynb`
4. `1.3-langchain-simple-agent.ipynb`
5. `2.0-autogen-building-agents.ipynb`
6. `2.1-autogen-building-research-assistant.ipynb`
7. `3.0-langgraph-building-stateful-applications.ipynb`
8. `3.1-langgraph-buid-research-assistant.ipynb`
9. `4.0-rag-agent-langgraph.ipynb`
10. `4.1-autogen-multi

{'input': 'What kind of files are in the current directory?',
 'output': 'The current directory contains the following files:\n\n1. `1.0-langchain-basics.ipynb`\n2. `1.1-langchain-simple-chains.ipynb`\n3. `1.2-langchain-runnables-and-lcel.ipynb`\n4. `1.3-langchain-simple-agent.ipynb`\n5. `2.0-autogen-building-agents.ipynb`\n6. `2.1-autogen-building-research-assistant.ipynb`\n7. `3.0-langgraph-building-stateful-applications.ipynb`\n8. `3.1-langgraph-buid-research-assistant.ipynb`\n9. `4.0-rag-agent-langgraph.ipynb`\n10. `4.1-autogen-multi-agent-workflows.ipynb`\n11. `Autogen Demo Notebook.ipynb`\n12. `Tool Use.ipynb`\n13. `agentic-rag-langgraph.ipynb`\n14. `assets-resources` (directory)'}

In [14]:
import os

os.environ["TAVILY_API_KEY"] = 'YOUR TAVILY API KEY'

In [16]:
from langchain_community.tools import TavilySearchResults

In [17]:
search_tool = TavilySearchResults(
    max_results = 5,
    search_depth="advanced",
    include_answer=True,
)

search_tool.invoke('What is the capital of France?')

'AttributeError("\'FieldInfo\' object has no attribute \'raw_results\'")'

In [18]:
@tool
def create_file(file_path: str, content: str):
    """Create a file with the given content"""
    with open(file_path, 'w') as f:
        f.write(content)
    return f"File created at {file_path} with content: {content}"

tools = [create_file, search_tool]

In [19]:
simple_research_agent = create_tool_calling_agent(llm, tools, prompt_agent)

simple_research_agent_executor = AgentExecutor(agent=simple_research_agent, tools=tools, verbose=True)

simple_research_agent_executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [20]:
from IPython.display import Markdown
output_report = simple_research_agent_executor.invoke({'input': 'Write a summarized report about the main highlights of the Paris Olympics 2024 and save it locally as report-olympics-2024.md'})

Markdown(output_report['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 highlights'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 highlights summary'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris 2024 Olympics main highlights'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 news highlights'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris 2024 Olympics key highlights'}`


AttributeError("'FieldInfo' object has no attribute 'raw_results'")
Invoking: `tavily_search_results_json` with `{'query': 'Paris 2024 Olympics overview'}`


AttributeError("'FieldInfo' object has no attri

Agent stopped due to max iterations.

In [21]:
with open('report-olympics-2024.md', 'r') as f:
    content_report = f.read()
    
Markdown(content_report)

FileNotFoundError: [Errno 2] No such file or directory: 'report-olympics-2024.md'